# Feature Engineering

## Libraries

In [1]:
import numpy as np
import pandas as pd
from cnr_methods import get_simplified_data, transform_data, LOFO_GPU_Importance

# Feature Engineering Library for Time Series
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table

# Feature Selection Libraries
from lofo import LOFOImportance, Dataset, plot_importance
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_absolute_error
import xgboost as xgb

## Read Data

For this pipeline, only Training Set will be used.

In [2]:
full_data = get_simplified_data()
full_data = full_data[full_data['Set']=='Train']
y_train = pd.read_csv('Data/Y_train.csv')

As done in the other Notebooks, we will transform the Column 'Time' to Datetime format and set as the index of the dataset.

In [3]:
full_data['Time'] = pd.to_datetime(full_data['Time'],dayfirst=True)
full_data = full_data.set_index('Time')

In [4]:
full_data.head()

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.44,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.26,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.707402,-0.853745,0.997093,-1.415138,287.00,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.78,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.46,95.905879,Train


To simplify the work, we will generate features for just one Wind Farm. When doing modelling, the features, as the models, will be generated for all Wind Farms separately.

In [5]:
WF = 'WF1'
data = full_data[full_data['WF']==WF]
y_train = y_train[y_train['ID'].isin(data['ID'])]

In [6]:
full_data

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440000,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260000,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.707402,-0.853745,0.997093,-1.415138,287.000000,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780000,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460000,95.905879,Train
...,...,...,...,...,...,...,...,...,...
2019-01-15 20:00:00,37371,WF6,-0.995550,-5.465200,0.645083,-0.911460,273.935000,0.000000,Train
2019-01-15 21:00:00,37372,WF6,-0.221900,-4.461200,0.430113,-0.701325,275.073746,0.000000,Train
2019-01-15 22:00:00,37373,WF6,-0.874850,-4.515750,0.123965,-0.696413,272.945000,0.000000,Train


## Feature Creation

First, using the Zonal and Meridional Components of Wind, the Magnitude and Direction of Wind Vector for 100m and 10m height.

### Wind Speed Vector

In [7]:
feature_data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]
feature_data['Wind Speed 100m'] = np.sqrt(feature_data['U_100m']**2 + feature_data['V_100m']**2)
feature_data['Wind Direction 100m'] = np.arctan(feature_data['V_100m']/feature_data['U_100m'])
feature_data['Wind Speed 10m'] = np.sqrt(feature_data['U_10m']**2 + feature_data['V_10m']**2)
feature_data['Wind Direction 10m'] = np.arctan(feature_data['V_10m']/feature_data['U_10m'])
feature_data = feature_data.drop(['U_100m','V_100m','U_10m','V_10m'],axis=1)

Changing Reference for Negative Angles:

In [8]:
feature_data['Wind Direction 100m'] = feature_data['Wind Direction 100m'].apply(lambda x: 360 + x if x < 0 else x)
feature_data['Wind Direction 10m'] = feature_data['Wind Direction 10m'].apply(lambda x: 360 + x if x < 0 else x)

Using Wind Speed and Direction instead of U and V, we will create some variables over the Numerical Variables from the simplified data.

In [9]:
features = ['T', 'CLCT', 'Wind Speed 100m','Wind Direction 100m', 'Wind Speed 10m', 'Wind Direction 10m']

### Time-Relative Variables

Here,  Values for Last Week and Month for each Numerical Feature are generated.

In [10]:
for column in features:
    feature_data[column + '_last_week'] = feature_data[column].shift(7)
    feature_data[column + '_last_month'] = feature_data[column].shift(30)

Now, Month and Quarter Statistics(Mean,Median,Variance) are generated:

In [11]:
feature_data['Month_Number'] = feature_data.index.month
feature_data['Quarter_Number'] = feature_data.index.quarter

In [12]:
# Month
mean_month = feature_data.groupby('Month_Number').mean()[features]
median_month = feature_data.groupby('Month_Number').median()[features]
variance_month = feature_data.groupby('Month_Number').var()[features]

# Quarter
mean_quarter = feature_data.groupby('Quarter_Number').mean()[features]
median_quarter = feature_data.groupby('Quarter_Number').median()[features]
variance_quarter = feature_data.groupby('Quarter_Number').var()[features]

In [13]:
# Month
mean_month.columns = mean_month.columns + '_Month_Mean'
median_month.columns = median_month.columns + '_Month_Median'
variance_month.columns = variance_month.columns + '_Month_Variance'

# Quarter
mean_quarter.columns = mean_quarter.columns + '_Quarter_Mean'
median_quarter.columns = median_quarter.columns + '_Quarterh_Median'
variance_quarter.columns = variance_quarter.columns + '_Quarter_Variance'

In [14]:
# Month
feature_data = feature_data.merge(mean_month,on='Month_Number',how='left')
feature_data = feature_data.merge(median_month,on='Month_Number',how='left')
feature_data = feature_data.merge(variance_month,on='Month_Number',how='left')

# Quarter
feature_data = feature_data.merge(mean_quarter,on='Quarter_Number',how='left')
feature_data = feature_data.merge(median_quarter,on='Quarter_Number',how='left')
feature_data = feature_data.merge(variance_quarter,on='Quarter_Number',how='left')

In [15]:
feature_data.index = data.index

For periodical Features, here represented by days (Of Month, Week and Year), hour and minutes, the features are applied to sinusoidal functions to replicate the cyclic nature of the variables.

In [16]:
day = feature_data.index.day
hour = feature_data.index.hour
minute = feature_data.index.minute
dayofweek = feature_data.index.dayofweek
dayofyear = feature_data.index.dayofyear

In [17]:
days_in_month = feature_data.index.days_in_month

In [18]:
feature_data["cos_day"], feature_data["sin_day"] = (
    np.cos(2 * np.pi * (day - 1) / days_in_month),
    np.sin(2 * np.pi * (day - 1) / days_in_month),
    )

feature_data["cos_hour"], feature_data["sin_hour"] = (
    np.cos(2 * np.pi * hour / 24),
    np.sin(2 * np.pi * hour / 24),
    )

feature_data["cos_minute"], feature_data["sin_minute"] = (
    np.cos(2 * np.pi * minute / 60),
    np.sin(2 * np.pi * minute / 60),
)

feature_data["cos_dayofyear"], feature_data["sin_dayofyear"] = (
    np.cos(2 * np.pi * (dayofyear - 1) / 365),
    np.sin(2 * np.pi * (dayofyear - 1) / 365),
)

feature_data["cos_dayofweek"], feature_data["sin_dayofweek"] = (
    np.cos(2 * np.pi * dayofweek / 7),
    np.sin(2 * np.pi * dayofweek / 7),
)

### Distance from Features

Distance of Position of Max and Min (Already on Tsfresh, check it later):

In [19]:
for column in features:
    feature_data[column + '_Distance_Max'] = feature_data.index - feature_data[column].idxmax()
    feature_data[column + '_Distance_Min'] = feature_data.index - feature_data[column].idxmin()
    feature_data[column + '_Distance_Max'] = feature_data[column + '_Distance_Max'].apply(lambda x : x.days)
    feature_data[column + '_Distance_Min'] = feature_data[column + '_Distance_Min'].apply(lambda x : x.days)

### Rolling Window Variables

### Wavelet Transformations (Check)

## Tsfresh

Now we use Tsfresh, a Python Library that automates Feature Engineering for Time Series Data. We generate new features for all the columns on the Simplified Data, as done below.

In [20]:
data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]

In [21]:
tsfresh_data = pd.DataFrame()
for variable in ['U_100m','V_100m','U_10m','V_10m','T','CLCT']: 
    df_shift, y = make_forecasting_frame(data[variable],kind=variable,max_timeshift=20,rolling_direction=1)
    X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,show_warnings=False,n_jobs=3)
    X['Feature'] = variable
    tsfresh_data = tsfresh_data.append(X)

Feature Extraction: 100%|██████████| 15/15 [01:38<00:00,  6.57s/it]


Process tsfresh_data to pass column 'Features' to the other columns

In [22]:
tsfresh_data = tsfresh_data.pivot(columns='Feature')

In [23]:
tsfresh_data.columns = tsfresh_data.columns.map('{0[0]}|{0[1]}'.format)

In [24]:
tsfresh_data.head()

,value__abs_energy|CLCT,value__abs_energy|T,value__abs_energy|U_100m,value__abs_energy|U_10m,value__abs_energy|V_100m,value__abs_energy|V_10m,value__absolute_sum_of_changes|CLCT,value__absolute_sum_of_changes|T,value__absolute_sum_of_changes|U_100m,value__absolute_sum_of_changes|U_10m,...,value__variance|U_100m,value__variance|U_10m,value__variance|V_100m,value__variance|V_10m,value__variance_larger_than_standard_deviation|CLCT,value__variance_larger_than_standard_deviation|T,value__variance_larger_than_standard_deviation|U_100m,value__variance_larger_than_standard_deviation|U_10m,value__variance_larger_than_standard_deviation|V_100m,value__variance_larger_than_standard_deviation|V_10m
id,,,,,,,,,,,,,,,,,,,,,
2018-05-01 02:00:00,6813.370572,82047.8736,5.055752,1.574030,10.613261,0.083919,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-01 03:00:00,16811.539518,163992.6612,10.982542,7.778652,12.704466,0.254793,17.447701,0.18,0.186000,1.236305,...,0.008649,0.382112,0.820564,0.003824,1.0,0.0,0.0,0.0,0.0,0.0
2018-05-01 04:00:00,26487.652359,246361.6612,13.897765,8.772845,13.433347,2.257409,19.071310,0.92,0.913098,2.730120,...,0.095117,0.425138,1.045847,0.253942,1.0,0.0,0.0,0.0,1.0,0.0
2018-05-01 05:00:00,35486.186521,327461.3096,27.635908,9.248391,52.089410,3.181777,22.577941,3.14,6.327000,3.037614,...,6.458769,0.467797,4.356622,0.202684,1.0,0.0,1.0,0.0,1.0,0.0
2018-05-01 06:00:00,44684.124127,408378.8012,42.177927,9.333069,81.733079,3.268781,23.623216,3.46,6.433900,3.436218,...,8.384555,0.556415,4.485969,0.198239,1.0,0.0,1.0,0.0,1.0,0.0


In [25]:
tsfresh_data = tsfresh_data.fillna(0)

In [26]:
tsfresh_data = tsfresh_data.loc[:, tsfresh_data.apply(pd.Series.nunique) != 1]

## Feature Selection

In [27]:
final_features = feature_data.merge(tsfresh_data,left_on=feature_data.index,right_on=tsfresh_data.index,how='left')

final_features = final_features.rename({'key_0':'Date'},axis=1)

In [28]:
features = final_features.drop(['ID','WF','Set','Date'],axis=1).columns

In [29]:
final_features = transform_data(final_features[features])
y_train = transform_data(y_train[['Production']])

In [34]:
final_features = final_features.fillna(0)

In [36]:
model  = xgb.XGBRegressor(tree_method='gpu_hist')

In [37]:
importance_df = LOFO_GPU_Importance(final_features,y_train['Production'],features,model)

1/2196 7.560465 s/it
2/2196 7.573887 s/it
3/2196 7.379378 s/it
4/2196 7.504586 s/it
5/2196 7.433017 s/it
6/2196 8.665843 s/it
7/2196 7.547362 s/it
8/2196 7.515623 s/it
9/2196 7.547223 s/it
10/2196 7.583317 s/it
11/2196 7.548256 s/it
12/2196 7.557456 s/it
13/2196 7.579090 s/it
14/2196 7.577536 s/it
15/2196 7.550996 s/it
16/2196 7.574998 s/it
17/2196 7.558925 s/it
18/2196 7.575335 s/it
19/2196 7.572879 s/it
20/2196 7.608207 s/it
21/2196 7.612769 s/it
22/2196 7.610843 s/it
23/2196 7.622132 s/it
24/2196 7.638321 s/it
25/2196 7.605008 s/it
26/2196 7.626520 s/it
27/2196 7.606869 s/it
28/2196 7.606030 s/it
29/2196 7.626378 s/it
30/2196 7.642647 s/it
31/2196 7.639409 s/it
32/2196 7.634527 s/it
33/2196 7.632740 s/it
34/2196 7.671117 s/it
35/2196 7.663697 s/it
36/2196 7.668099 s/it
37/2196 7.710511 s/it
38/2196 7.676333 s/it
39/2196 7.695557 s/it
40/2196 7.683718 s/it
41/2196 7.688845 s/it
42/2196 7.694269 s/it
43/2196 7.706395 s/it
44/2196 7.768304 s/it
45/2196 7.785336 s/it
46/2196 7.805467 s/

KeyboardInterrupt: 

In [32]:
importance_df.to_excel('Importance_DF_WF1.xlsx')